# Examples of Using the TaskSolver Component

The `TaskSolver` class provides a reusable and parameterizable interface for running prompt-based tasks with LLMs. It wraps an `LLMPromptProcessor` and a template prompt (defined as a list of `Message` objects with placeholders), allowing dynamic substitution of parameters or message content at runtime.

It supports various input types - including text, image URLs, audio files, and more - depending on the capabilities of the underlying LLM. Once initialized with a task definition (e.g., a translation instruction), you can call `.solve()` asynchronously (and `solve_batch()`) with either a dictionary of values - that contains the value of the replacements in the message - or a complete `Message` to generate a response.

Use cases include:
- Translating text rows in a DataFrame
- Describing images via image URLs or uploads
- Transcribing audio inputs
- Structuring unstructured texts

`TaskSolver` is a building block of agent-like LLM applications.

In [2]:
# Auto reload imports
%reload_ext autoreload
%autoreload 2

# Visualization
%matplotlib inline

# Imports
import pandas as pd
import os

## Translating a DataFrame column to English
To initialize the `TaskSolver`, you can provide a list of `Message` objects—typically including one or more system messages and optionally a user message. If no user message is provided at initialization, you can supply it later when calling the `solve()` method.

Prompts can be defined as a list of dictionaries, where each dictionary corresponds to a message. Placeholders (e.g., `{sentence}`, `{destination_language}`) can be included in the content and will be automatically replaced during prompt formatting. For example, `{sentence}` should match a column name in your DataFrame if you’re applying the solver row-wise.

In [4]:
example_df = pd.DataFrame({'sentence': [
    "A mélytanulás a gépi tanulás egyik ága, amely a neurális hálózatokat használja.",
    "深度学习是机器学习的一个分支，它使用神经网络。",
    "L'apprentissage profond est une branche de l'apprentissage automatique qui utilise des réseaux de neurones.",
]})

example_df.head(2)

,sentence
0,"A mélytanulás a gépi tanulás egyik ága, amely ..."
1,深度学习是机器学习的一个分支，它使用神经网络。


In [5]:
from sziaaitls.components.llm_agents import TaskSolver
from sziaaitls.core.llms import LLMPromptProcessor, Message

prompt_list = [
    {'role': 'system', 'content': 'You are a robot that translates sentences to {destination_language}.'},
    {'role': 'user', 'content': 'Translate the following sentence: {sentence}'}
]
formatted_prompt_list = [Message.from_dict(p).format(destination_language="English") for p in prompt_list]

task_solver = TaskSolver(
    llm=LLMPromptProcessor(
        api_key=os.environ.get("OPENAI_API_KEY"),
        model="gpt-4o-mini",
        rate_limit=350,
        period=60,
    ),
    base_messages=formatted_prompt_list,
)

In [6]:
translations = await task_solver.solve_batch(
    input_list=example_df[['sentence']].to_dict(orient='records'),
)

example_df['translation'] = [t.content for t in translations]
example_df

,sentence,translation
0,"A mélytanulás a gépi tanulás egyik ága, amely ...",Deep learning is a branch of machine learning ...
1,深度学习是机器学习的一个分支，它使用神经网络。,Deep learning is a branch of machine learning ...
2,L'apprentissage profond est une branche de l'a...,Deep learning is a branch of machine learning ...


## Address Cleaning

**F.I.N.**